In [4]:
from glob import glob
import pandas as pd
import os
import nbformat
from functools import reduce

In [5]:
book2 = glob("../../notebooks/book2/*/*.ipynb")
len(book2), book2[0]

(182, '../../notebooks/book2/18/bnn_mnist_sgld.ipynb')

In [23]:
a = "GAN"
a.isupper()

True

In [34]:
def upper_to_capitalize(nb):
     return "_".join(list(map(lambda word: word.capitalize() if word.isupper() else word, nb.split("_"))))
    
print(upper_to_capitalize("GAN_loss_types.ipynb"))
print(upper_to_capitalize("GAN_JAX_CelebA_demo.ipynb"))
print(upper_to_capitalize("two_moons_normalizingFlow.ipynb"))

Gan_loss_types.ipynb
Gan_Jax_CelebA_demo.ipynb
two_moons_normalizingFlow.ipynb


In [37]:
edge_cases={
"gan__jax__celeb_a_demo.ipynb": "gan__jax__celebA__demo.ipynb",
"dirac_g_a_n.ipynb" : "dirac_gan.ipynb",
"l_v_a_e.ipynb": "lvae.ipynb",
"sts__t_f_p.ipynb": "sts.ipynb",
"v_i_b_demo2021.ipynb":"vib_demo_2021.ipynb"
}

def change_case(camel_nb_path):
    camel_nb = camel_nb_path.split("/")[-1]
    
    #handle GAN_loss -> Gan_loss otherwise it would give g_a_n_loss
    snake_nb = upper_to_capitalize(camel_nb) 
    snake_nb = reduce(lambda x, y: x + ('_' if y.isupper() else '') + y, snake_nb).lower()
    if snake_nb in edge_cases:
        snake_nb = edge_cases[snake_nb]
    
    snake_nb_path = camel_nb_path.replace(camel_nb, snake_nb)
    if snake_nb_path != camel_nb_path:
        return (camel_nb_path, snake_nb_path)

In [40]:
old_new_nb = list(filter(lambda x: x,list(map(change_case, book2))))
print(len(old_new_nb))
for (old, new) in old_new_nb:
    print(new)

18
../../notebooks/book2/03/eb_binom.ipynb
../../notebooks/book2/29/newsgroups_visualize.ipynb
../../notebooks/book2/17/gpr_demo_noise_free.ipynb
../../notebooks/book2/17/gpr_demo_ard.ipynb
../../notebooks/book2/17/gp_kernel_plot.ipynb
../../notebooks/book2/17/gpr_demo_change_hparams.ipynb
../../notebooks/book2/25/gan_loss_types.ipynb
../../notebooks/book2/25/gan__jax__celebA__demo.ipynb
../../notebooks/book2/25/ipm_divergences.ipynb
../../notebooks/book2/25/dirac_gan.ipynb
../../notebooks/book2/06/em_log_likelihood_max.ipynb
../../notebooks/book2/14/probit_reg_demo.ipynb
../../notebooks/book2/20/lvae.ipynb
../../notebooks/book2/28/sts.ipynb
../../notebooks/book2/05/vib_demo_2021.ipynb
../../notebooks/book2/27/mix__ppca_celeba.ipynb
../../notebooks/book2/27/mix_ppca_demo.ipynb
../../notebooks/book2/22/two_moons_normalizing_flow.ipynb


## Rename these notebooks

In [50]:
def rename_nb(pair):
    old, new = pair
    os.rename(old, new)

In [51]:
list(map(rename_nb, old_new_nb))

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

## Update latex source code

In [41]:
tex_files = glob("../../../bookv2/book2/*/*.tex")
tex_files[5], len(tex_files)

('../../../bookv2/book2/comp/comp-methods.tex', 384)

In [58]:
def make_latex_nb(nb_path):
    """
    foo_jax -> foo\_jax
    """
    nb = nb_path.split("/")[-1]
    nb = nb.replace("_", "\_").replace(".ipynb", "")
    return nb


def replace_keyword_in_tex(tex_file, old_nb, new_nb):

    with open(tex_file, "r") as fp:
        content = fp.read()

    old = "\\notebook{" + make_latex_nb(old_nb)  # foo_jax -> foo\_jax
    new = "\\notebook{" + make_latex_nb(new_nb)

    if old in content:
        print(f"In {tex_file} - {old} -> {new}")
    content = content.replace(old, new)

    with open(tex_file, "w") as fp:
        fp.write(content)

In [59]:
# replace_keyword_in_tex(tex_files[5],"advi_beta_binom_jax.ipynb","advi_beta_binom.ipynb")

In [60]:
for (old, new) in old_new_nb:
    list(map(replace_keyword_in_tex, tex_files, [old] * len(tex_files), [new] * len(tex_files)))

In ../../../bookv2/book2/stats/eb.tex - \notebook{ebBinom -> \notebook{eb\_binom
In ../../../bookv2/book2/info/relnet.tex - \notebook{newsgroupsVisualize -> \notebook{newsgroups\_visualize
In ../../../bookv2/book2/gp/gp-regression.tex - \notebook{gprDemoNoiseFree -> \notebook{gpr\_demo\_noise\_free
In ../../../bookv2/book2/gp/kernels.tex - \notebook{gprDemoArd -> \notebook{gpr\_demo\_ard
In ../../../bookv2/book2/gp/kernels.tex - \notebook{gpKernelPlot -> \notebook{gp\_kernel\_plot
In ../../../bookv2/book2/gp/gp-representationlearning.tex - \notebook{gprDemoChangeHparams -> \notebook{gpr\_demo\_change\_hparams
In ../../../bookv2/book2/gan/algorithms_theory.tex - \notebook{GAN\_loss\_types -> \notebook{gan\_loss\_types
In ../../../bookv2/book2/gan/algorithms_theory.tex - \notebook{IPM\_divergences -> \notebook{ipm\_divergences
In ../../../bookv2/book2/gan/algorithms_theory.tex - \notebook{DiracGAN -> \notebook{dirac\_gan
In ../../../bookv2/book2/opt/em.tex - \notebook{emLogLikelihoodMax 

## Update in notebooks 

In [61]:
book2 = glob("../../notebooks/book2/*/*.ipynb")
len(book2), book2[0]

(182, '../../notebooks/book2/18/bnn_mnist_sgld.ipynb')

In [62]:
def apply_fun_to_notebook(notebook, fun, old_nb_path, new_nb_path):
    """
    fun should take one argument: code
    """
    nb = nbformat.read(notebook, as_version=4)
    for cell in nb.cells:
        code = cell["source"]
        updated_code = fun(code, old_nb_path, new_nb_path)
        if updated_code != code:
            cell["source"] = updated_code
            nbformat.write(nb, notebook)


def replace_keyword(code, old_nb_path, new_nb_path):
    old_nb = old_nb_path.split("/")[-1]
    new_nb = new_nb_path.split("/")[-1]
    code = code.replace(old_nb, new_nb)
    return code

In [63]:
book2 = glob("../../notebooks/book2/*/*.ipynb")
for pair in old_new_nb:
    list(
        map(
            apply_fun_to_notebook, book2, [replace_keyword] * len(book2), [pair[0]] * len(book2), [pair[1]] * len(book2)
        )
    )